In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os

import pandas as pd
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras

Mounted at /content/drive


In [ ]:
path = './drive/My Drive/MasterIADeporte/TFM/'

In [ ]:
df_report = pd.read_csv(path+"complete_report_benzema.csv")
df_report['Fwd_cont'] = 1 # [1, 0, 1, 1, 0, 0, 1, 0.5, 0.8, 0.8, 0.8, 0.8, 
                          # 0.8, 0.8, 0.5, 0.2, 0.2, 0.2, 0.2, 0.2]
df_report['Mid_cont'] = 1 # [1, 0, 1, 1, 0, 0, 1, 0.5, 0.8, 0.8, 0.8, 0.8, 
                          # 0.8, 0.8, 0.5, 0.2, 0.2, 0.2, 0.2, 0.2]
df_report['Def_cont'] = 1 # [1, 0, 1, 1, 0, 0, 1, 0.5, 0.8, 0.8, 0.8, 0.8, 
                          # 0.8, 0.8, 0.5, 0.2, 0.2, 0.2, 0.2, 0.2]
fwd_sum = sum(df_report.Percentile*df_report.Fwd_cont)
mid_sum = sum(df_report.Percentile*df_report.Mid_cont)
def_sum = sum(df_report.Percentile*df_report.Def_cont)
df_final = pd.DataFrame({'Statistic': ['Fwd_Label', 'Mid_Label', 'Def_Label'], 
            'Per 90': [fwd_sum, mid_sum, def_sum], 
            'Percentile': [int(round(fwd_sum/sum(df_report.Fwd_cont), 0)),
                           int(round(mid_sum/sum(df_report.Mid_cont), 0)),
                           int(round(def_sum/sum(df_report.Def_cont), 0))], 
            'Fwd_cont': [0, 0, 0],  'Mid_cont': [0, 0, 0], 
            'Def_cont': [0, 0, 0]})
df_report = df_report.append(df_final, ignore_index=True)
df_report

,Statistic,Per 90,Percentile,Fwd_cont,Mid_cont,Def_cont
0,Goals,0.81,97,1,1,1
1,Assists,0.24,84,1,1,1
2,Non-Penalty Goals,0.75,97,1,1,1
3,Penalty Kicks Made,0.07,66,1,1,1
4,Penalty Kicks Attempted,0.07,58,1,1,1
...,...,...,...,...,...,...
147,Aerials lost,1.30,92,1,1,1
148,% of Aerials Won,39.2%,60,1,1,1
149,Fwd_Label,9912,67,0,0,0
150,Mid_Label,9912,67,0,0,0


In [ ]:
df_summary = pd.read_csv(path+"summary_logs_benzema.csv")
df_passing = pd.read_csv(path+"passing_logs_benzema.csv")
df_passtypes = pd.read_csv(path+"passtypes_logs_benzema.csv")
df_goals = pd.read_csv(path+"goalshotcreation_logs_benzema.csv")
df_def = pd.read_csv(path+"def_logs_benzema.csv")
df_possesion = pd.read_csv(path+"possesion_logs_benzema.csv")
df_misc = pd.read_csv(path+"miscstats_logs_benzema.csv")
key = ['Player', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 
       'Opponent', 'Start', 'Pos', 'Min']
df = df_summary[key + ['Gls', 'PK', 'PKatt', 'Sh', 'SoT', 'xG', 'npxG']].merge(
                     df_passing, left_on=key, right_on=key)
df = df.merge(df_passtypes, left_on=key, right_on=key)
df = df.merge(df_goals, left_on=key, right_on=key)
df = df.merge(df_def, left_on=key, right_on=key)
df = df.merge(df_possesion, left_on=key, right_on=key)
df = df.merge(df_misc[key + ['CrdY', 'CrdR', '2CrdY', 'FlsComm', 'FlsDrawn', 
                       'Offsides', 'PKwon', 'PKcon', 'OG', 'Recov', 
                       'AerialDuelsWon', 'AerialDuelsLost', 
                       'AerialDuelsWinPerc', 'Match Report']], 
              left_on=key, right_on=key)
df = df.fillna(0)
df['Start'] = df['Start'].apply(lambda x: 1 if x in ('Y', 'Y*') else 0)
df['Opponent'] = df['Opponent'].apply(lambda x: x[2:] if x[0].islower() else x)
df['Squad'] = df['Squad'].apply(lambda x: x[2:] if x[0].islower() else x)
df['Result'] = df['Result'].apply(lambda x: x.split(' '))
df['Score'] = df['Result'].apply(lambda x: x[1])
df['Result'] = df['Result'].apply(lambda x: x[0])
df['Score'] = df['Score'].apply(lambda x: x.split('–'))
df['Squad_Goals'] = df['Score'].apply(lambda x: x[0])
df['Opponent_Goals'] = df['Score'].apply(lambda x: x[1])
df = df.drop(columns=['Score', 'Match Report_x', 'Match Report_y'])
df

,Player,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Gls,PK,PKatt,Sh,SoT,xG,npxG,TotCmp,TotAtt,TotCmpPerc,TotDist,PrgDist,ShCmp,ShAtt,ShCmpPerc,MedCmp,MedAtt,MedCmpPerc,LongCmp,LongAtt,LongCmpPerc,Ast,xA,KP,1/3,PPA,CrsPA,Prog,...,Err,Touches,DefPenTouches,Def3rdTouches,Mid3rdTouches,Att3rdTouches,AttPenTouches,LiveTouches,SuccDrib,AttDrib,SuccDribPerc,NuPlDrib,Nutmegs,Carries,TotDistCarries,PrgDistCarries,ProgCarries,1/3Carries,CPA,MisCarries,DisCarries,TargPass,RecPass,RecPassPerc,ProgPassRec,CrdY,CrdR,2CrdY,FlsComm,FlsDrawn,Offsides,PKwon,PKcon,OG,Recov,AerialDuelsWon,AerialDuelsLost,AerialDuelsWinPerc,Squad_Goals,Opponent_Goals
0,Karim Benzema,2021-08-14,Sat,La Liga,Matchweek 1,Away,W,Real Madrid,Alavés,1,FW,88,2,0,0,6,3,0.7,0.7,39.0,45.0,86.7,720.0,130.0,23.0,25.0,92.0,7.0,8.0,87.5,9.0,9.0,100.0,0,0.0,0.0,2.0,1.0,0.0,4.0,...,0.0,55.0,0.0,3.0,25.0,30.0,10.0,53.0,2.0,2.0,100.0,2.0,0.0,36.0,194.0,78.0,5.0,2.0,2.0,0.0,1.0,58.0,48.0,82.8,10.0,0,0,0,0,1,1,0.0,0.0,0,3.0,1.0,0.0,100.0,4,1
1,Karim Benzema,2021-08-22,Sun,La Liga,Matchweek 2,Away,D,Real Madrid,Levante,1,FW,90,0,0,0,1,0,0.1,0.1,27.0,32.0,84.4,365.0,67.0,19.0,23.0,82.6,8.0,8.0,100.0,0.0,0.0,0.0,2,0.2,2.0,0.0,3.0,0.0,3.0,...,0.0,39.0,0.0,0.0,17.0,23.0,8.0,35.0,0.0,0.0,0.0,0.0,0.0,25.0,126.0,44.0,3.0,1.0,0.0,1.0,2.0,44.0,29.0,65.9,5.0,0,0,0,1,1,4,0.0,0.0,0,5.0,0.0,0.0,0.0,3,3
2,Karim Benzema,2021-08-28,Sat,La Liga,Matchweek 3,Away,W,Real Madrid,Betis,1,FW,90,0,0,0,2,0,0.1,0.1,31.0,37.0,83.8,389.0,92.0,18.0,20.0,90.0,9.0,11.0,81.8,1.0,2.0,50.0,1,0.3,3.0,0.0,4.0,1.0,5.0,...,0.0,48.0,0.0,4.0,21.0,27.0,4.0,47.0,2.0,2.0,100.0,2.0,1.0,30.0,112.0,74.0,5.0,2.0,2.0,5.0,0.0,48.0,37.0,77.1,3.0,0,0,0,0,1,3,0.0,0.0,0,6.0,1.0,2.0,33.3,1,0
3,Karim Benzema,2021-09-01,Wed,WCQ — UEFA (M),First round,Home,D,France,Bosnia and Herzegovina,1,FW,75,0,0,0,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0.0,0.0,0,0.0,0.0,0.0,0.0,1,1
4,Karim Benzema,2021-09-04,Sat,WCQ — UEFA (M),First round,Away,D,France,Ukraine,0,0,27,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0.0,0,0.0,0.0,0.0,0.0,1,1
5,Karim Benzema,2021-09-07,Tue,WCQ — UEFA (M),First round,Home,W,France,Finland,1,FW,90,0,0,0,5,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,2,0,0.0,0.0,0,0.0,0.0,0.0,0.0,2,0
6,Karim Benzema,2021-09-12,Sun,La Liga,Matchweek 4,Home,W,Real Madrid,Celta Vigo,1,FW,90,3,1,1,4,2,1.1,0.3,39.0,44.0,88.6,601.0,140.0,22.0,23.0,95.7,11.0,13.0,84.6,4.0,4.0,100.0,1,1.3,4.0,3.0,1.0,0.0,5.0,...,0.0,55.0,0.0,5.0,20.0,31.0,9.0,49.0,1.0,1.0,100.0,1.0,0.0,36.0,160.0,89.0,5.0,1.0,0.0,2.0,1.0,67.0,44.0,65.7,11.0,0,0,0,2,0,1,0.0,0.0,0,2.0,0.0,1.0,0.0,5,2
7,Karim Benzema,2021-09-15,Wed,Champions Lg,Group stage,Away,W,Real Madrid,Inter,1,FW,90,0,0,0,2,0,0.1,0.1,33.0,43.0,76.7,575.0,97.0,20.0,23.0,87.0,6.0,10.0,60.0,6.0,7.0,85.7,0,0.3,2.0,3.0,3.0,0.0,4.0,...,0.0,51.0,1.0,4.0,22.0,29.0,6.0,51.0,1.0,1.0,100.0,1.0,0.0,41.0,196.0,118.0,6.0,3.0,1.0,3.0,2.0,59.0,45.0,76.3,6.0,0,0,0,0,0,1,0.0,0.0,0,5.0,2.0,1.0,66.7,1,0
8,Karim Benzema,2021-09-19,Sun,La Liga,Matchweek 5,Away,W,Real Madrid,Valencia,1,FW,90,1,0,0,5,3,0.5,0.5,29.0,34.0,85.3,413.0,125.0,16.0,18.0,88.9,11.0,12.0,91.7,1.0,2.0,50.0,1,0.1,1.0,2.0,2.0,0.0,5.0,...,0.0,41.0,0.0,0.0,20.0,24.0,6.0,39.0,2.0,2.0,100.0,2.0,0.0,35.0,183.0,157.0,9.0,3.0,2.0,1.0,0.0,58.0,37.0,63.8,5.0,0,0,0,1,0,1,0.0,0.0,0,1.0,1.0,1.0,50.0,2,1
9,Karim Benzema,2021-09-22,Wed,La Liga,Matchweek 6,Home,W,Real Madrid,Mallorca,1,FW,79,2,0,0,4,3,0.6,0.6,34.0,44.0,77.3,506.0,87.0,21.0,28.0,75.0,12.0,14.0,85.7,1.0,2.0,